In [1]:
from tensorflow.keras.datasets import cifar100
from hyperopt import fmin, hp, tpe, space_eval
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Activation, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as k
from sklearn.preprocessing import Normalizer
import numpy as np
from tqdm import tqdm

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# scaler = Normalizer()

                    
# for i,img in tqdm(enumerate(x_train)):
#     for j in range(3):
#         x_train[i,:,:,j] = scaler.fit_transform(img[:,:,j])
        
# for i,img in tqdm(enumerate(x_test)):
#     for j in range(3):
#         x_test[i,:,:,j] = scaler.fit_transform(img[:,:,j])

x_train = np.array([(img - img.min()) / (img.max() - img.min()) for img in x_train])
x_test = np.array([(img - img.min()) / (img.max() - img.min()) for img in x_test])                   

In [3]:
def objective(space):
    l = space['l']
    lr = space['lr']
    m_batch = space['m_batch']
    ep_batch = space['ep_batch']
    dr = space['dr']
    model = space['model']
    x_train = space['x_train']
    y_train = space['y_train']
    x_test = space['x_test']
    y_test = space['y_test']
    val_loss = 100
    
    if model == 1:
        val_loss = model1(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test)
    elif model == 2:
        val_loss = model2(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test)
    elif model == 3:
        val_loss = model3(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test)
    else:
        val_loss = model4(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test)
        
    return val_loss

def model1(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test):
    
    model = Sequential()
    
    model.add( Conv2D(32, (5,5), padding='same', input_shape=(32,32,3)))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(64, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(96, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(128, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(160, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(192, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Flatten())
    
    model.add( Dense(1024))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(100))
    model.add( Activation('softmax'))
    
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy', metrics=['accuracy'])
    
    r = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=0)
    
    evaluate = model.evaluate(x_test, y_test, batch_size=100, verbose=0)
    
    k.clear_session()
    del model
    
    return evaluate[0]
    
    
def model2(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test):
    
    model = Sequential()
    
    model.add( Conv2D(32, (5,5), padding='same', input_shape=(32,32,3)))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(64, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(96, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(128, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(160, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Conv2D(192, (5,5), padding='same', kernel_regularizer=l2(l=l)))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    
    model.add( Conv2D(256, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    
    model.add( Flatten())
    
    model.add( Dense(1024))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(100))
    model.add( Activation('softmax'))
    
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy', metrics=['accuracy'])
    
    r = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=0)
    
    evaluate = model.evaluate(x_test, y_test, batch_size=100, verbose=0)
    
    k.clear_session()
    del model
    
    return evaluate[0]

def model3(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test):
    model = Sequential()
    
    model.add( Conv2D(32, (5,5), padding='same', input_shape=(32,32,3)))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(64, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(96, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(128, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(160, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(192, (5,5), padding='same', kernel_regularizer=l2(l=l)))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    
    model.add( Conv2D(224, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(256, (5,5)))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(512, (5,5), padding='same'))
    model.add( Activation('relu'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Flatten())
    
    model.add( Dense(1024))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(100))
    model.add( Activation('softmax'))
    
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy', metrics=['accuracy'])
    
    r = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=0)
    
    evaluate = model.evaluate(x_test, y_test, batch_size=100, verbose=0)
    
    k.clear_session()
    del model
    
    return evaluate[0]

def model4(l, lr, m_batch, ep_batch, dr, x_train, y_train, x_test, y_test):
    model = Sequential()
    
    model.add( Conv2D(32, (3,3), padding='same', input_shape=(32,32,3)))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(64, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(96, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(128, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(160, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(192, (3,3), padding='same', kernel_regularizer=l2(l=l)))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    
    model.add( Conv2D(224, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(256, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Conv2D(512, (3,3), padding='same'))
    model.add( Activation('relu'))
    model.add( MaxPool2D(padding='same'))
    model.add(BatchNormalization(momentum=m_batch, epsilon=ep_batch))
    
    model.add( Flatten())
    
    model.add( Dense(2024))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(1024))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(512))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(256))
    model.add( Activation('relu'))
    model.add( Dropout(dr))
    
    model.add( Dense(100))
    model.add( Activation('softmax'))
    
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy', metrics=['accuracy'])
    
    r = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=0)
    
    evaluate = model.evaluate(x_test, y_test, batch_size=100, verbose=0)
    
    k.clear_session()
    del model
    
    return evaluate[0]
 

In [4]:
# l = space['l']
# lr = space['lr']
# m_batch = space['m_batch']
# ep_batch = space['ep_batch']
# dr = space['dr']
# model = space['model']
# val_loss = 100
space = {'l': hp.uniform('l', 0.01, 0.09),
         'lr': hp.uniform('lr', 0.0001, 0.009),
         'm_batch': hp.uniform('m_batch', 0.1, 0.99),
         'ep_batch': hp.uniform('ep_batch', 0.1, 0.99),
         'dr': hp.uniform('dr', 0.1, 0.5),
         'model': hp.choice('model', [1,2,3,4]),
         'x_train': x_train,
         'y_train': y_train,
         'x_test': x_test,
         'y_test': y_test
        }

best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [13:25:47<00:00, 483.47s/trial, best loss: 2.4441186273097992]  


In [5]:
best

{'dr': 0.22908672852156392,
 'ep_batch': 0.5058885669537324,
 'l': 0.0827965533776402,
 'lr': 0.0002430161533396024,
 'm_batch': 0.5037201398959942,
 'model': 0}